In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time
import pandas as pd
import math
import random
import pickle

In [2]:
# # Open the input file and output file in the same context
# with open('llmrawdata.txt', 'r') as infile, open('lmprocesseddata.txt', 'w') as outfile:
#     for line in infile:
#         if 'l4s_ecn_marking-start' in line:
#             # Remove the prefix 'l4s_ecn_marking-start,' and trailing ' end'
#             temp=line.split("-")
#             temp1=temp[1]
            
#             temp1=temp1.split(",")
#             #print(temp1[-1]=="end \n")
#             if(temp1[-1]=="end \n"):
#                 outfile.writelines(temp[1])


In [3]:
# Open the input file and output file in the same context
with open('llmrawdata.txt', 'r') as infile, open('lmprocesseddata.txt', 'w') as outfile:
    for line in infile:
        if 'l4s_ecn_marking-start' in line:
            # Remove the prefix 'l4s_ecn_marking-start,' and trailing ' end'
            parts = line.split("-")
            if len(parts) > 1:
                content = parts[1].strip()  # Remove any leading/trailing whitespace
                if content.endswith('end'):
                    # Write the content to the output file, stripping the 'end' part
                    outfile.write(content[6:-4] + '\n')


In [4]:
column_list = [
    "queue_type",                   # q->queue_type
    "qdelay_reference",             # pprms->qdelay_ref
    "tupdate",                      # pprms->tupdate
    "max_burst",                    # pprms->max_burst
    "max_ecn_threshold",            # pprms->max_ecnth
    "alpha_coefficient",            # pprms->alpha
    "beta_coefficient",             # pprms->beta
    "flags",                        # pprms->flags
    "burst_allowance",              # pst->burst_allowance
    "drop_probability",             # pst->drop_prob
    "current_queue_delay",          # pst->current_qdelay
    "previous_queue_delay",         # pst->qdelay_old
    "accumulated_probability",      # pst->accu_prob
    "measurement_start_time",       # pst->measurement_start
    "average_dequeue_time",         # pst->avg_dq_time
    "dequeue_count",                # pst->dq_count
    "status_flags",                 # pst->sflags
    "total_packets",                # q->stats.tot_pkts
    "total_bytes",                  # q->stats.tot_bytes
    "queue_length",                 # q->stats.length
    "length_in_bytes",              # q->stats.len_bytes
    "total_drops",                  # q->stats.drops
    "dequeue_action",               # dequeue_action
]


In [5]:
df=pd.read_csv("lmprocesseddata.txt",names=column_list,header=None)

# Drop columns that contain 'pprms' in their name
columns_to_drop = [
    "qdelay_reference",             # pprms->qdelay_ref
    "tupdate",                      # pprms->tupdate
    "max_burst",                    # pprms->max_burst
    "max_ecn_threshold",            # pprms->max_ecnth
    "alpha_coefficient",            # pprms->alpha
    "beta_coefficient",             # pprms->beta
    "flags",                        # pprms->flags
    ]
df = df.drop(columns=columns_to_drop)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 721858 entries, 0 to 721857
Data columns (total 16 columns):
 #   Column                   Non-Null Count   Dtype
---  ------                   --------------   -----
 0   queue_type               721858 non-null  int64
 1   burst_allowance          721858 non-null  int64
 2   drop_probability         721858 non-null  int64
 3   current_queue_delay      721858 non-null  int64
 4   previous_queue_delay     721858 non-null  int64
 5   accumulated_probability  721858 non-null  int64
 6   measurement_start_time   721858 non-null  int64
 7   average_dequeue_time     721858 non-null  int64
 8   dequeue_count            721858 non-null  int64
 9   status_flags             721858 non-null  int64
 10  total_packets            721858 non-null  int64
 11  total_bytes              721858 non-null  int64
 12  queue_length             721858 non-null  int64
 13  length_in_bytes          721858 non-null  int64
 14  total_drops              721858 non-

In [7]:
df.columns.to_list()

['queue_type',
 'burst_allowance',
 'drop_probability',
 'current_queue_delay',
 'previous_queue_delay',
 'accumulated_probability',
 'measurement_start_time',
 'average_dequeue_time',
 'dequeue_count',
 'status_flags',
 'total_packets',
 'total_bytes',
 'queue_length',
 'length_in_bytes',
 'total_drops',
 'dequeue_action']

In [8]:
df.shape[0] *0.2

144371.6

In [9]:

class ExperiencePool:
    """
    Experience pool for collecting trajectories.
    """
    def __init__(self):
        self.states = []
        self.actions = []
        self.rewards = []
        self.dones = []

    def add(self, state, action, reward, done):
        self.states.append(state)  # sometime state is also called obs (observation)
        self.actions.append(action)
        self.rewards.append(reward)
        self.dones.append(done)

    def __len__(self):
        return len(self.states)

# Define the list of columns to include
columns_to_use = [
    'queue_type', 
    'burst_allowance',
    'drop_probability',
    'current_queue_delay',
    'accumulated_probability',
    'average_dequeue_time',
    'total_bytes',
    'total_drops'
]


import pickle
exp_pool = ExperiencePool()
# Initialize the global reward variable
global_reward = 0

# Iterate through each row and update the global reward variable
for index, row in df.iterrows():
    # print(row[columns_to_use])
    state = np.array(row[columns_to_use], dtype=np.float32)
    print(type(state))
    exp_pool.add(state=state, action=row['dequeue_action'], reward=row['current_queue_delay'], done=0)
    break;

pickle_save_path='exp_pool_l4s.pkl'
pickle.dump(exp_pool, open( pickle_save_path, 'wb'))
print(f"Done. Experience pool saved at:", pickle_save_path)

<class 'numpy.ndarray'>
Done. Experience pool saved at: exp_pool_l4s.pkl


In [10]:

class ExperiencePool:
    """
    Experience pool for collecting trajectories.
    """
    def __init__(self):
        self.states = []
        self.actions = []
        self.rewards = []
        self.dones = []

    def add(self, state, action, reward, done):
        self.states.append(state)  # sometime state is also called obs (observation)
        self.actions.append(action)
        self.rewards.append(reward)
        self.dones.append(done)

    def __len__(self):
        return len(self.states)

# Define the list of columns to include
columns_to_use = [
    'queue_type', 
    'burst_allowance',
    'drop_probability',
    'current_queue_delay',
    'accumulated_probability',
    'average_dequeue_time',
    'total_bytes',
    'total_drops'
]


import pickle
exp_pool = ExperiencePool()
# Initialize the global reward variable
global_reward = 0

# Iterate through each row and update the global reward variable
for index, row in df.iterrows():
    state = np.array(row[columns_to_use], dtype=np.float32)
    exp_pool.add(state=state, action=row['dequeue_action'], reward=row['current_queue_delay'], done=0)
    if index > df.shape[0] *0.2:
        break;

pickle_save_path='exp_pool_l4s.pkl'
pickle.dump(exp_pool, open( pickle_save_path, 'wb'))
print(f"Done. Experience pool saved at:", pickle_save_path)

Done. Experience pool saved at: exp_pool_l4s.pkl


In [11]:
type(exp_pool.states)

list

In [12]:
type(exp_pool.states[0])

numpy.ndarray

In [13]:
exp_pool.states[0].dtype

dtype('float32')

In [14]:
type(exp_pool.actions)

list

In [15]:
type(exp_pool.rewards)

list

In [16]:
exp_pool.states

[array([0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32),
 array([0.0e+00, 1.5e+05, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 6.0e+01,
        0.0e+00], dtype=float32),
 array([0.0e+00, 1.5e+05, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 1.2e+02,
        0.0e+00], dtype=float32),
 array([     0., 150000.,      0.,      0.,      0.,      0.,    180.,
             0.], dtype=float32),
 array([     0., 150000.,      0.,  10000.,      0.,      0.,    240.,
             0.], dtype=float32),
 array([     0., 150000.,      0.,      0.,      0.,      0.,    300.,
             0.], dtype=float32),
 array([     0., 150000.,      0.,  10000.,      0.,      0.,    360.,
             0.], dtype=float32),
 array([     0., 150000.,      0.,      0.,      0.,      0.,    420.,
             0.], dtype=float32),
 array([     0., 150000.,      0.,  10000.,      0.,      0.,    480.,
             0.], dtype=float32),
 array([1., 0., 0., 0., 0., 0., 0., 0.], dtype=float32),
 array([     0., 150000.,      0.,      0.,   